In [17]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset

In [18]:
class Net(torch.nn.Module):
 def __init__(self):
     super(Net, self).__init__()
     self.conv1 = torch.nn.Sequential(torch.nn.Conv2d(1, 32, 3, 1, 1),torch.nn.ReLU(),torch.nn.MaxPool2d(2))
     self.conv2 = torch.nn.Sequential(torch.nn.Conv2d(32, 64, 3, 1, 1),torch.nn.ReLU(),torch.nn.MaxPool2d(2))
     self.conv3 = torch.nn.Sequential(torch.nn.Conv2d(64, 64, 3, 1, 1),torch.nn.ReLU(),torch.nn.MaxPool2d(2))
     self.dense = torch.nn.Sequential(torch.nn.Linear(64 * 3 * 3, 128),torch.nn.ReLU(),torch.nn.Linear(128, 10))
 
 def forward(self, x):
     conv1_out = self.conv1(x)
     conv2_out = self.conv2(conv1_out)
     conv3_out = self.conv3(conv2_out)
     res = conv3_out.view(conv3_out.size(0), -1)
     out = self.dense(res)
     return out

In [19]:
epsilons=[0,.05,.1,.15,.2,.25,.3]
#pretrained_model="/content/drive/MyDrive/MNIST_small_cnn.checkpoint"
use_cuda=True

In [20]:
dataset=datasets.MNIST(root='/content/drive/MyDrive/MNIST', train=False, download=False, transform=transforms.Compose([transforms.ToTensor(),]))
data=torch.tensor(dataset.data[:10000].float()).unsqueeze(1)
data=(data-data.min())/(data.max()-data.min())
train_ds=TensorDataset(data,dataset.targets[:10000])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [21]:
test_loader=torch.utils.data.DataLoader(train_ds,batch_size=1,shuffle=True)
device=torch.device("cuda")
model=Net().to(device)
checkpoint=torch.load('/content/drive/MyDrive/TrustworthyML/model.pth',map_location=lambda storage,loc:storage)
model.load_state_dict(checkpoint)
model.eval()

Net(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=576, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [22]:
def fgsm_attack(image,epsilon,data_grad):
  sign_data_grad=data_grad.sign()
  perturbed_image=image+epsilon*sign_data_grad
  perturbed_image=torch.clamp(perturbed_image,0,1)
  return perturbed_image

In [23]:
def test(model,device,test_loader,epsilon):
    #Accuracy Counter
    correct=0
    adv_examples=[]

    for data,target in test_loader:
        data,target=data.to(device),target.to(device)
        data.requires_grad=True

        output=model(data)
        init_pred=output.max(1,keepdim=True)[1]
        if init_pred.item()!=target.item():
            continue

        loss=F.nll_loss(output,target)
        model.zero_grad()
        loss.backward()
        data_grad=data.grad.data

        perturbed_data=fgsm_attack(data,epsilon,data_grad)
        output=model(perturbed_data)
        final_pred=output.max(1,keepdim=True)[1]
        if final_pred.item()==target.item():
            correct+=1
            if (epsilon==0) and (len(adv_examples)<5):
                adv_ex=perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append((init_pred.item(),final_pred.item(),adv_ex))
        else:
            if len(adv_examples)<5:
                adv_ex=perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append((init_pred.item(),final_pred.item(),adv_ex))

    final_acc=correct/float(len(test_loader))
    success_rate=float(len(test_loader)-correct)/float(len(test_loader))
    print("Epsilon: {}\tTest Accuracy={}/{}={}\tSuccess Rate={}".format(epsilon,correct,len(test_loader),final_acc,success_rate))
    return final_acc,adv_examples

In [ ]:
accuracies=[]
examples=[]

for eps in epsilons:
  acc,ex=test(model,device,test_loader,eps)
  accuracies.append(acc)
  examples.append(ex)

Epsilon: 0	Test Accuracy=9907/10000=0.9907	Success Rate=0.0093
Epsilon: 0.05	Test Accuracy=9638/10000=0.9638	Success Rate=0.0362
Epsilon: 0.1	Test Accuracy=8961/10000=0.8961	Success Rate=0.1039
Epsilon: 0.15	Test Accuracy=7659/10000=0.7659	Success Rate=0.2341
Epsilon: 0.2	Test Accuracy=5534/10000=0.5534	Success Rate=0.4466
Epsilon: 0.25	Test Accuracy=3365/10000=0.3365	Success Rate=0.6635


In [ ]:
temp=[1,1,1,1,1,1,1]
final=[0,0,0,0,0,0,0]
for i in range(7):
  final[i]=temp[i]-accuracies[i]

plt.figure(figsize=(10,10))
plt.plot(epsilons,final,"*-")
plt.yticks(np.arange(0,1.1,step=0.1))
plt.xticks(np.arange(0,.35,step=0.05))
plt.title("Epsilon-Success Rate")
plt.xlabel("Epsilon")
plt.ylabel("Success Rate")
plt.show()

In [ ]:
cnt=0
plt.figure(figsize=(8,10))
for i in range(len(epsilons)):
  for j in range(len(examples[i])):
    cnt+=1
    plt.subplot(len(epsilons),len(examples[0]),cnt)
    plt.xticks([],[])
    plt.yticks([],[])
    if j==0:
      plt.ylabel("Eps: {}".format(epsilons[i]),fontsize=14)
    orig,adv,ex=examples[i][j]
    plt.title("{} -> {}".format(orig,adv))
    plt.imshow(ex,cmap="gray")
plt.tight_layout()
plt.show()